In [45]:
! pip install sseclient
!pip install time

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [46]:
#Filter Function will return user data after removing the duplicates in last five minutes 
def filter_user(final_user_list,user_dict):
    if((len(final_user_list)<1)and len(user_dict)<1):
        return final_user_list
    else:
        for i in user_dict:
            for j in final_user_list:
                if(j.get(i)!=None):
                    if(j.get(i)>user_dict.get(i)):
                        user_dict[i]=j.get(i)
                    j.pop(i)
        return final_user_list

In [47]:
#This function will print the values and returns the list of dictinary holding the user data of last five minutes 
def User_report(user_dict,user_list):
    temp={}
    #to check the 
    user_list=filter_user(user_list,user_dict)
    if(len(user_list)<5):
        user_list.append(user_dict)
    else:
        user_list.pop(0)
        user_list.append(user_dict)
    for i in user_list:
        for j in i:
            temp[j]=i.get(j)
    print("Users who made changes to en.wikipedia.org")
    s_list=sorted(temp.items(), key=lambda x: x[1], reverse=True)
    try:  
        for i in s_list:
            print("{}: {} ".format(i[0],i[1]),end="\n")
    except AttributeError:
        print("Empty List")
    print("\n")
    return user_list

In [48]:
#Filter Function will return domain data after removing the duplicates in last five minutes 
import copy
def domain_filter(final_domain_list,domain_dict):
    #creating the copy of dictinary 
    temp_domain_dict=copy.deepcopy(domain_dict) 
    # to check if the len is lees than 1
    if((len(final_domain_list)<1)and len(domain_dict)<1):
        return final_domain_list
    else:
        #if not then taking the union if two same domain occured
        for i in temp_domain_dict:
            for j in final_domain_list:
                if(j.get(i)!=None):
                    x=j.get(i)
                    y=temp_domain_dict.get(i)
                    j[i]=x.union(y)
                    domain_dict.pop(i)
        return final_domain_list

In [49]:
#This fucntion ensures that count of unique page tittles and print the domain report 
def domain_report1(domain_dict,domain_list):
    domain_list=domain_filter(domain_list,domain_dict)
    temp={}
    #to check if the data is having less than 5 minutes 
    if(len(domain_list)<5):
        #print(templist,"list")
        domain_list.append(domain_dict)
    else:
        # if not then popping the first minute data 
        domain_list.pop(0)
        #print(templist,"list")
        domain_list.append(domain_dict)
        # counting the unique tittles
    for i in domain_list:
        for j in i:
            temp[j]=len(i[j])
    #sorting the values 
    sorted_list=sorted(temp.items(), key=lambda x: x[1], reverse=True)
    #printing the values 
    print("Total number of Wikipedia Domains Updated: {} \n \n".format(len(sorted_list)))
    for i in sorted_list:
        print("{}: {} pages updated".format(i[0],i[1]))
    print("\n \n")
    return domain_list
        
        

In [50]:
# This will create the list of page tittle for each minute 
def Validate_Domain(change,temp):
    # Assign the set variable to have the unique page tittle 
    test_list=set()
    #check for the vlaue none 
    if(change.get("page_title")!=None):
        #to check if we have exist domain
            if(temp.get(change.get('meta').get("domain"))==None):
                test_list.add(change.get("page_title"))
                temp[change.get('meta').get("domain")]=test_list
            else:
                #if not then creating the domain and assigning the value 
                test_list=temp[change.get('meta').get("domain")]
                test_list.add(change.get("page_title"))
                temp[change.get('meta').get("domain")]=test_list
    return temp
    

In [51]:
#This will filter all the domains for en.wikipedia.org and user is not bot and create the list for one minute 
def Validate_user(temp,change):
    #Check for the condition if value is none 
    if(temp.get(change.get('performer').get("user_text"))==None and change.get('performer').get("user_text")!= None):
        try:
            #check for the domain en.en.wikipedia.org and the user is not bot
            if(change.get('meta').get("domain")=="en.wikipedia.org" and change.get('performer').get("user_is_bot")!=True):
                #if edit count is none then assigning value 0
                if(change.get('performer').get("user_edit_count")==None):
                    temp[change.get('performer').get("user_text")]=0
                else:
                    #if not then assigning the edit count
                    temp[change.get('performer').get("user_text")]=change.get('performer').get("user_edit_count")
        except AttributeError:
            temp[change.get('performer').get("user_text")]=0
    return temp

In [52]:
# This function will load the json, create the process to run parallely and generate the report of each domain and user 
import json
import time
from sseclient import SSEClient as EventSource
import multiprocessing as mp
def export_data(url):
    # manager dictonary will have the user data for 60 min which is being validated
    validate_dict=mp.Manager().dict()
    # dictionary of the json
    change={}
    # manager dictonary will have the domain data for 60 min which is being validated
    temp=mp.Manager().dict()
    #list containing the domain data of last 5 minutes with the count of unique values
    final_temp=list()
    # Declare to track the time for 60 seconds
    current=time.time()
    final_list=list()#list containing the user data of last 5 minutes 
    for event in EventSource(url):
        if event.event == 'message':
            try:
                change = json.loads(event.data)
            except ValueError: 
                pass
            if(len(change)!=0):
                #Created the process to call Validate_user
                p1=mp.Process(target=Validate_user(validate_dict,change))
                #The process has started 
                p1.start()
                #The process has terminated 
                p1.terminate()
                #Created the process to call Validate_domain
                p2=mp.Process(target=Validate_Domain(change,temp))
                #The Process has started 
                p2.start()
                #The process has terminate 
                p2.terminate()
                #sleep to get process terminated 
                time.sleep(0.2)
                # check for the time of 60 seconds 
                if(current+60<=time.time()):
                    final_list=User_report(validate_dict,final_list)
                    final_temp=domain_report1(temp,final_temp)
                    #Cleared the one minute data after storing 
                    validate_dict={}
                    temp={}
                    #assign the current time to iterate for the next 60 minutes
                    current=time.time()

In [53]:
if __name__ == "__main__":
    #Calling the function to call the url
    try:
        export_data('https://stream.wikimedia.org/v2/stream/revision-create')
    except KeyboardInterrupt:
        print("Program Terminated")

Users who made changes to en.wikipedia.org
Simeon: 324524 
FeanorStar7: 239739 
Robby.is.on: 99113 
GraemeLeggett: 99048 
Mztourist: 59310 
Kahtar: 35429 
Francis Flinch: 31388 
Joseywales1961: 25996 
Njd-de: 15666 
Rchard2scout: 13964 
Heshs Umpire: 4541 
Ivaneurope: 3397 
Martimix: 3124 
CrafterNova: 1645 
Ruairidhbevan: 1136 
Alexander Doria: 1103 
Malig77: 180 
123.203.105.62: 0 
2409:4071:248A:6DDF:0:0:6FB:8B0: 0 
2A02:C7F:5CD6:4D00:C538:6922:826E:5487: 0 
42.106.237.161: 0 
86.161.225.49: 0 


Total number of Wikipedia Domains Updated: 42 
 

commons.wikimedia.org: 75 pages updated
www.wikidata.org: 47 pages updated
en.wikipedia.org: 37 pages updated
ja.wikipedia.org: 10 pages updated
de.wikipedia.org: 10 pages updated
ca.wiktionary.org: 6 pages updated
tr.wikipedia.org: 6 pages updated
fr.wikipedia.org: 6 pages updated
it.wikipedia.org: 5 pages updated
ru.wikipedia.org: 5 pages updated
meta.wikimedia.org: 4 pages updated
arz.wikipedia.org: 4 pages updated
en.wiktionary.org: 3 pa